In [ ]:
# Clarke-Wright Savings Heuristic for Vehicle Routing Problem with Simultaneous Pickup and Delivery (VRPSPD)
# Savings-based VRPSPD (Gajpal & Abad style)

costMatrix = [
 [0,9,14,23,32,50,21,49,30,27,35,28,18],
 [9,0,21,22,36,52,24,51,36,37,41,30,20],
 [14,21,0,25,38,5,31,7,36,43,29,7,6],
 [23,22,25,0,42,12,35,17,44,31,31,11,6],
 [32,36,38,42,0,22,37,16,46,37,29,13,14],
 [50,52,5,12,22,0,41,23,10,39,9,17,16],
 [21,24,31,35,37,41,0,26,21,19,10,25,12],
 [49,51,7,17,16,23,26,0,30,28,16,27,12],
 [30,36,36,44,46,10,21,30,0,25,22,10,20],
 [27,37,43,31,37,39,19,28,25,0,20,16,8],
 [35,41,29,31,29,9,10,16,22,20,0,10,10],
 [28,30,7,11,13,17,25,27,10,16,10,0,10],
 [18,20,6,6,14,16,12,12,20,8,10,10,0]
]

demand = [0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700, 1100]
pickup = [1100, 0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700]
capacityOfVehicle = 6000
numberOfVehicles = 4

def route_distance(route):
    """Distance for a route (route = list of customer indices, depot index = 0)."""
    if not route:
        return 0
    d = costMatrix[0][route[0]]
    for a, b in zip(route, route[1:]):
        d += costMatrix[a][b]
    d += costMatrix[route[-1]][0]
    return d

def total_delivery(route):
    return sum(demand[i] for i in route)

def feasibility_check_route(route, Q):
    """
    VRPSPD feasibility via cumulative net-pickup:
      initial load = total deliveries for route (vehicle leaves depot carrying all deliveries)
      traverse route: load = load - delivery + pickup
      check 0 <= load <= Q at every step
    Returns: (feasible: bool, max_load_seen, final_load)
    """
    if not route:
        return True, 0, 0
    Dr = total_delivery(route)
    load = Dr
    if load > Q:
        return False, load, load
    max_load = load
    for i in route:
        load = load - demand[i] + pickup[i]
        if load < 0:
            return False, max_load, load
        if load > max_load:
            max_load = load
        if load > Q:
            return False, max_load, load
    return True, max_load, load

# --- Initialize routes: one per customer ---
customers = list(range(1, len(costMatrix)))  # 1..12
routes = [[i] for i in customers]

# --- Compute savings ---
savings = []
for i in customers:
    for j in customers:
        if j <= i:
            continue
        s = costMatrix[0][i] + costMatrix[0][j] - costMatrix[i][j]
        savings.append(((i, j), s))
savings.sort(key=lambda x: x[1], reverse=True)

def find_route_index(routes, cust):
    for idx, r in enumerate(routes):
        if cust in r:
            return idx
    return None

# --- Process savings ---
for (i, j), s in savings:
    ri = find_route_index(routes, i)
    rj = find_route_index(routes, j)
    if ri is None or rj is None or ri == rj:
        continue
    route_i = routes[ri]
    route_j = routes[rj]
    # endpoints only
    if not (i == route_i[0] or i == route_i[-1]):
        continue
    if not (j == route_j[0] or j == route_j[-1]):
        continue
    new_route = None
    # four possible concatenations to keep endpoints adjacency
    if i == route_i[-1] and j == route_j[0]:
        cand = route_i + route_j
        feasible, max_load, final = feasibility_check_route(cand, capacityOfVehicle)
        if feasible:
            new_route = cand
    if new_route is None and i == route_i[0] and j == route_j[-1]:
        cand = route_j + route_i
        feasible, max_load, final = feasibility_check_route(cand, capacityOfVehicle)
        if feasible:
            new_route = cand
    if new_route is None and i == route_i[0] and j == route_j[0]:
        cand = list(reversed(route_i)) + route_j
        feasible, max_load, final = feasibility_check_route(cand, capacityOfVehicle)
        if feasible:
            new_route = cand
    if new_route is None and i == route_i[-1] and j == route_j[-1]:
        cand = route_i + list(reversed(route_j))
        feasible, max_load, final = feasibility_check_route(cand, capacityOfVehicle)
        if feasible:
            new_route = cand
    if new_route is not None:
        # merge (remove higher index first)
        if ri > rj:
            del routes[ri]
            del routes[rj]
        else:
            del routes[rj]
            del routes[ri]
        routes.append(new_route)

# --- Output ---
print("Final routes (customer indices):")
total_dist = 0.0
for idx, r in enumerate(routes, 1):
    d = route_distance(r)
    total_dist += d
    feas, max_load, final_load = feasibility_check_route(r, capacityOfVehicle)
    print(f" Route {idx}: {r}  | Distance = {d:.1f} | Feasible = {feas} | Max_load_seen = {max_load} | Total_delivery = {total_delivery(r)}")

print(f"\nNumber of routes: {len(routes)} (Vehicles available: {numberOfVehicles})")
print(f"Total distance (all routes): {total_dist:.1f}")
if len(routes) > numberOfVehicles:
    print("WARNING: result needs more vehicles than available!")

print("\nRoutes with depot (0) shown:")
for idx, r in enumerate(routes, 1):
    print(f" Route {idx}: {[0] + r + [0]}")


Final routes (customer indices):
 Route 1: [10, 5, 7]  | Distance = 116.0 | Feasible = True | Max_load_seen = 4700 | Total_delivery = 4500
 Route 2: [8, 11, 4]  | Distance = 85.0 | Feasible = True | Max_load_seen = 5000 | Total_delivery = 5000
 Route 3: [1, 3, 12, 9]  | Distance = 72.0 | Feasible = True | Max_load_seen = 5600 | Total_delivery = 5600
 Route 4: [2, 6]  | Distance = 66.0 | Feasible = True | Max_load_seen = 3100 | Total_delivery = 3100

Number of routes: 4 (Vehicles available: 4)
Total distance (all routes): 339.0

Routes with depot (0) shown:
 Route 1: [0, 10, 5, 7, 0]
 Route 2: [0, 8, 11, 4, 0]
 Route 3: [0, 1, 3, 12, 9, 0]
 Route 4: [0, 2, 6, 0]
